<a href="https://colab.research.google.com/github/vincent0965/Colab_note/blob/master/poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ml0930/raw/master/poem_train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ml0930/raw/master/poem_test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
train_df

In [ ]:
test_df = pd.read_csv("test.csv", encoding="utf-8")
test_df

In [ ]:
# Series.replace({"李白":0})
# unique/value_counts
writers = train_df["作者"].unique()
trans = {w:i for i, w in enumerate(writers)}
trans_r = {i:w for i, w in enumerate(writers)}
# 維度轉換
y_train = train_df["作者"].replace(trans)
y_test = test_df["作者"].replace(trans)
y_test

In [ ]:
# 標點符號要去掉, \r \n 都要去掉
import jieba
p = train_df["內容"][0]
" ".join(jieba.cut(p))

In [ ]:
# apply(函式名字)
def poemcut(p):
    return " ".join(jieba.cut(p))

# 在pandas表格中執行函式(例如poemcut)
x_train = train_df["內容"].apply(poemcut)
x_test = test_df["內容"].apply(poemcut)
x_test

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.fit_transform(x_test)

In [ ]:
x_train_count
# 2731是指有2731首詩；52294是指有出現52294個欄位

In [ ]:
print(x_train_count)
# 只儲存非0的欄位

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
vec.vocabulary_

In [39]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.1)
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [40]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

0.8333333333333334

In [41]:

from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
      index=[name + "(原本)" for name in writers],
      columns=[name + "(預測)" for name in writers],)

,李白(預測),杜甫(預測),白居易(預測)
李白(原本),8,1,1
杜甫(原本),0,9,1
白居易(原本),1,1,8


In [42]:
p = input("輸入一首詩:")
x_predict = vec.transform([poemcut(p)])
proba = clf.predict_proba(x_predict)[0]
for w, p in zip(writers, proba):
    print(w, ":", p)

輸入一首詩:嗨嗨嗨嗨
李白 : 0.35481508604906625
杜甫 : 0.4236543390699377
白居易 : 0.22153057488099598
